# Adaptive

In [ ]:
import holoviews as hv
from holoviews.streams import Stream, param
hv.notebook_extension('bokeh')

In [ ]:
import numpy as np
import adalearner
from time import sleep
from random import randint
from functools import partial
import ipyparallel
import concurrent.futures
import importlib
importlib.reload(adalearner)

In [ ]:
import asyncio
from ipykernel.eventloops import register_integration

@register_integration('asyncio')
def loop_asyncio(kernel):
    '''Start a kernel with asyncio event loop support.'''
    loop = asyncio.get_event_loop()

    def kernel_handler():
        loop.call_soon(kernel.do_one_iteration)
        loop.call_later(kernel._poll_interval, kernel_handler)

    loop.call_soon(kernel_handler)
    try:
        if not loop.is_running():
            loop.run_forever()
    finally:
        loop.run_until_complete(loop.shutdown_asyncgens())
        loop.close()

In [ ]:
%gui asyncio

In [ ]:
def func(x, wait=True):
    """Function with a sharp peak on a smooth background"""
    import numpy as np
    from time import sleep
    x = np.asarray(x)
    a = 0.001
    if wait:
        sleep(np.random.randint(1, 3))
    return x + a**2/(a**2 + (x)**2)

# Parallel

In [ ]:
learner = adalearner.Learner1D(func, client=ipyparallel.Client())

learner.add_point(-1, func(-1))
learner.add_point(1, func(1))

In [ ]:
learner.start()

In [ ]:
data_stream = Stream.define('data', data=param.ObjectSelector(default=dict()))
dm = hv.DynamicMap(learner.plot, streams=[data_stream()])
dm

In [ ]:
async def monitor(delay=1):
    while True:
        dm.event(data=learner.data)
        await asyncio.sleep(delay)
        
monitor_task = learner.ioloop.create_task(monitor())

In [ ]:
learner.task.print_stack()